In [1]:
import os
import shutil
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
os.listdir('drive')

['Shareddrives',
 '.file-revisions-by-id',
 'MyDrive',
 '.shortcut-targets-by-id',
 '.Trash-0',
 '.Encrypted']

In [12]:
# Paths
source_dir = "drive/MyDrive/Rice_Leaf_AUG/Rice_Leaf_AUG/Rice_Leaf_AUG"  # Replace with your main folder path

In [13]:
os.listdir(source_dir)

['Rice Hispa',
 'Brown Spot',
 'Sheath Blight',
 'Neck_Blast',
 'Leaf scald',
 'Narrow Brown Leaf Spot',
 'Leaf Blast',
 'Healthy Rice Leaf',
 'Bacterial Leaf Blight']

In [14]:


# Paths
  # Replace with your main folder path
destination_dir = "drive/MyDrive/Rice_Leaf_AUG1/Rice_Leaf_AUG1/Rice_Leaf_AUG1"  # Replace with your new folder path

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Get only the image files (modify extensions if needed)
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    if image_files:  # If there are images in the folder
        # Randomly select half of the images
        num_to_copy = len(image_files) // 2
        images_to_copy = random.sample(image_files, num_to_copy)

        # Create the corresponding subdirectory in the destination folder
        relative_path = os.path.relpath(root, source_dir)
        dest_subdir = os.path.join(destination_dir, relative_path)
        os.makedirs(dest_subdir, exist_ok=True)

        # Copy the selected images
        for image in images_to_copy:
            src_path = os.path.join(root, image)
            dest_path = os.path.join(dest_subdir, image)
            shutil.copy2(src_path, dest_path)  # Use copy2 to preserve metadata

print("Images have been copied successfully.")


Images have been copied successfully.


In [15]:
os.listdir(destination_dir)

['Rice Hispa',
 'Brown Spot',
 'Sheath Blight',
 'Neck_Blast',
 'Leaf scald',
 'Narrow Brown Leaf Spot',
 'Leaf Blast',
 'Healthy Rice Leaf',
 'Bacterial Leaf Blight']

In [16]:
import os
import shutil
import random

# Paths
source_dir = "drive/MyDrive/Rice_Leaf_AUG1/Rice_Leaf_AUG1/Rice_Leaf_AUG1"  # Replace with your dataset path
output_dir = "drive/MyDrive/RiceDatatest"  # Replace with your output path

# Ratios for splitting
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Ensure output directories exist
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Split the dataset
for root, dirs, files in os.walk(source_dir):
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    if image_files:  # If there are images in the folder
        random.shuffle(image_files)
        num_images = len(image_files)
        train_end = int(train_ratio * num_images)
        val_end = train_end + int(val_ratio * num_images)

        splits = {
            'train': image_files[:train_end],
            'val': image_files[train_end:val_end],
            'test': image_files[val_end:]
        }

        for split, split_files in splits.items():
            split_dir = os.path.join(output_dir, split, os.path.relpath(root, source_dir))
            os.makedirs(split_dir, exist_ok=True)
            for image in split_files:
                shutil.copy2(os.path.join(root, image), os.path.join(split_dir, image))

print("Dataset split into train, val, and test sets.")


Dataset split into train, val, and test sets.


In [3]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
data_dir = "drive/MyDrive/RiceDatatest"
train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

# Dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Load DenseNet121
model = models.densenet121(pretrained=True)
num_features = model.classifier.in_features
model.classifier = torch.nn.Linear(num_features, len(train_data.classes))  # Adjust for your number of classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 96.5MB/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [6]:
model.summary

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

# Define optimizer, loss, and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
epochs = 20
best_val_accuracy = 0

print("Starting training...")
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model.train()
    train_loss = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Track progress of training
        if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
            print(f"  Batch {batch_idx+1}/{len(train_loader)}: Loss {loss.item():.4f}")

    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1} Training Complete: Average Loss: {train_loss:.4f}")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(val_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader)
    val_accuracy = correct / total
    print(f"Validation Complete: Average Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pth")
        print(f"  New best model saved with Accuracy: {best_val_accuracy:.4f}")

    scheduler.step()
    print(f"Learning rate adjusted to: {scheduler.get_last_lr()[0]:.6f}")
print("Training complete!")


Starting training...
Epoch 1/20
  Batch 10/129: Loss 0.7442
  Batch 20/129: Loss 0.4479
  Batch 30/129: Loss 0.5638
  Batch 40/129: Loss 0.7677
  Batch 50/129: Loss 0.5592
  Batch 60/129: Loss 0.3348
  Batch 70/129: Loss 0.7676
  Batch 80/129: Loss 0.4879
  Batch 90/129: Loss 0.6663
  Batch 100/129: Loss 0.2799
  Batch 110/129: Loss 0.4670
  Batch 120/129: Loss 0.7048
  Batch 129/129: Loss 0.6669
Epoch 1 Training Complete: Average Loss: 0.6338
Validation Complete: Average Loss: 0.5444, Accuracy: 0.8126
  New best model saved with Accuracy: 0.8126
Learning rate adjusted to: 0.001000
Epoch 2/20
  Batch 10/129: Loss 0.6804
  Batch 20/129: Loss 0.4645
  Batch 30/129: Loss 0.4132
  Batch 40/129: Loss 0.7912
  Batch 50/129: Loss 0.4422
  Batch 60/129: Loss 0.5051
  Batch 70/129: Loss 0.6202
  Batch 80/129: Loss 0.4063
  Batch 90/129: Loss 0.3499
  Batch 100/129: Loss 0.2972
  Batch 110/129: Loss 0.4387
  Batch 120/129: Loss 0.4877
  Batch 129/129: Loss 0.3798
Epoch 2 Training Complete: Avera

In [8]:
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy}")


<ipython-input-8-47d1599234d7>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Test Accuracy: 0.9716193656093489
